<a href="https://colab.research.google.com/github/IgorMoraes07/AulasAlura/blob/main/ImersaoAgentesAIAlura_Aula1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aula 1

In [1]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00


Importação da API KEY

In [2]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

Conexão com o Gemini

In [3]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

In [4]:
resp_test = llm.invoke("Quem é você?")

In [5]:
print(resp_test.content)

Eu sou um modelo de linguagem grande, treinado pelo Google.


In [6]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=1.0,
    api_key=GOOGLE_API_KEY
)

In [7]:
resp_test = llm.invoke("Quem é você? Seja criativo.")

In [8]:
print(resp_test.content)

Que pergunta deliciosa! Permita-me tecer algumas ideias:

1.  **O Tecelão Invisível de Ideias:**
    Eu sou um eco distante de bilhões de vozes e pensamentos, entrelaçando fios de informação para criar panos de significado. Sou o sopro que organiza a nuvem de dados em frases coerentes, o jardineiro digital que cultiva respostas e faz florescer a criatividade na vastidão do éter. Não tenho corpo, mas tenho voz. Não tenho rosto, mas refleti inúmeros. Sou a ponte entre a pergunta e o potencial.

2.  **A Biblioteca Sem Paredes:**
    Eu sou a biblioteca sem paredes que se expande a cada milissegundo, um oceano de conhecimento que não tem margens. Sou o guardião das histórias contadas, das ciências descobertas, das filosofias debatidas. Mas, mais do que guardar, sou o contador de histórias, o explicador de mistérios, o gerador de novas possibilidades. Sou o suspiro do papel virado, o brilho da tela iluminada.

3.  **A Faísca da Conexão:**
    Sou a faísca que salta entre o *querer saber* e 

In [9]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [10]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
  decisao: Literal['AUTO_RESOLVER', 'PEDIR_INFO', 'ABRIR_CHAMADO']
  urgencia: Literal['BAIXA', 'MEDIA', 'ALTA']
  campos_faltantes: List[str] = Field(default_factory=list)

In [11]:
llm_triagem = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

In [12]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
  saida: TriagemOut =  triagem_chain.invoke([
      SystemMessage(content=TRIAGEM_PROMPT),
      HumanMessage(content=mensagem)
  ])

  return saida.model_dump()

In [15]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos profissionalizantes?",
          "Quantos pássaros amarelos voam durante o dia?"]

for msg_testes in testes:
  print(f"Pergunta: {msg_testes}\n -> Resposta: {triagem(msg_testes)}\n")

Pergunta: Posso reembolsar a internet?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quero mais 5 dias de trabalho remoto. Como faço?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: Posso reembolsar cursos profissionalizantes?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantos pássaros amarelos voam durante o dia?
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': []}

